# AutoMPC Demo

Welcome!  This notebook demonstrates the core features of AutoMPC.  We will use the Cart-Pole swing-up task as an example, and we will tune an MPC pipeline consisting of a multi-layer perception (MLP) system ID model and an iLQR optimizer.

## Set-Up

To begin, we need an input dataset and specifications for the system and task.  The `benchmarks` module provides these for a few example systems.

In [1]:
import autompc as ampc
import numpy as np
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()


# Get system and task specification
system = benchmark.system
task   = benchmark.task

# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=500, traj_len=200)

Loading AutoMPC...
Cannot import SnoptWrapper
Cannot import KnitroSolver, make sure its Python interface is installed
Finished loading AutoMPC
running build_ext


Next, we need to declare our MPC pipeline.  The following code initializes a pipeline with a MLP system ID model, a quadratic objective function, and and an iLQR optimizer.

In [2]:
from autompc.sysid import MLPFactory
from autompc.control import IterativeLQRFactory
from autompc.costs import QuadCostFactory

model_factory = MLPFactory(system)
ctrlr_factory = IterativeLQRFactory(system)
cost_factory  = QuadCostFactory(system)

pipeline = ampc.Pipeline(system, model_factory, ctrlr_factory, cost_factory)

We can view the joint configuration space of the pipeline

In [3]:
pipeline.get_configuration_space()

Configuration space object:
  Hyperparameters:
    _cost:dx_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:dx_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _ctrlr:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    _model:hidden_size_1, Type: UniformInteger, Range: [16, 256], Default: 128
   

## Tuning

Tune with selected surrogate model class, default model configuration.

In [4]:
from autompc.tuning import PipelineTuner

surrogate_factory = MLPFactory(system)
tuner = PipelineTuner(surrogate_mode="defaultcfg", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100))

/usr/lib/python3/dist-packages/pyparsing.py:1745: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )
/home/william/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


use_cuda=True
MLP Using Cuda
hidden_sizes= [128, 128]
100%|██████████| 50/50 [01:23<00:00,  1.67s/it]

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-29_19:41:20_790610
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent



use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:40<00:00,  2.00s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:11<00:00, 16.55it/s]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 200.0000


...


Surrogate Cost:  200.0
Surrogate Final State:  [3.07377729 0.03142495 2.93310611 0.69010844]
use_cuda=True
MLP Using Cuda
hidden_sizes= [238, 128]
100%|██████████| 50/50 [01:25<00:00,  1.72s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:18<00:00, 10.90it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 104.038280 sec, available: 0.000010 sec)



Surrogate Cost:  63.0
Surrogate Final State:  [-0.00941103 -0.00449236 -0.13345293  0.00105161]
use_cuda=True
MLP Using Cuda
hidden_sizes= [255, 128]
100%|██████████| 50/50 [01:27<00:00,  1.74s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:19<00:00, 10.37it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 106.163151 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 4
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 12363.24 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 12229.17 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  _cost:dx_F, Value: 33.6356525523436
  _cost:dx_Q, Value: 0.0013622084435760135
  _cost:omega_F, Value: 1.3519710652415837
  _cost:omega_Q, Value: 0.01869440877657144



Surrogate Cost:  74.0
Surrogate Final State:  [-0.00711933  0.0564036  -0.14651211  0.08580299]
use_cuda=True
MLP Using Cuda
hidden_sizes= [238, 128]
100%|██████████| 50/50 [01:26<00:00,  1.74s/it]


We can also set the configuration of the surrogate model.

In [5]:
surrogate_cfg = surrogate_factory.get_configuration_space().get_default_configuration()
surrogate_cfg["n_hidden_layers"] = "2"
surrogate_cfg["hidden_size_1"] = 128
surrogate_cfg["hidden_size_2"] = 128       
surrogate_cfg["nonlintype"] = "relu"

tuner = PipelineTuner(surrogate_mode="fixedcfg", surrogate_factory=surrogate_factory,
                     surrogate_cfg=surrogate_cfg, surrogate_split=0.5)

controller, tune_result2 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

/home/william/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


use_cuda=True
MLP Using Cuda
hidden_sizes= [128, 128]
100%|██████████| 50/50 [01:25<00:00,  1.71s/it]

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-29_23:10:17_436873
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent



use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:46<00:00,  2.12s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:12<00:00, 15.95it/s]
Surrogate Cost:  200.0
Surrogate Final State:  [ 3.16447529e+00 -1.86618074e-03  4.48800437e-02 -3.67070618e-02]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:12<00:00, 16.06it/s]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 200.0000


...


True Dynamics Cost:  47.0
True Dynamics Final State:  [-1.20087772e-08  1.45236835e-06 -1.29121852e-01  2.78932314e-06]
use_cuda=True
MLP Using Cuda
hidden_sizes= [33, 145, 106, 119]
100%|██████████| 50/50 [02:19<00:00,  2.79s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:33<00:00,  5.84it/s]
Surrogate Cost:  185.0
Surrogate Final State:  [-0.01189215 -0.00475219 -0.22674637 -0.03755996]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:33<00:00,  5.99it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 206.389781 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 5



True Dynamics Cost:  200.0
True Dynamics Final State:  [ 3.22417292  0.77115759 -1.20651987 -3.40561783]


INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 18483.55 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 18323.56 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  _cost:dx_F, Value: 5915.448051127327
  _cost:dx_Q, Value: 0.7824060412248923
  _cost:omega_F, Value: 0.004640107866457608
  _cost:omega_Q, Value: 0.011887198551763617
  _cost:theta_F, Value: 9.393324497426475
  _cost:theta_Q, Value: 0.0010332840967745137
  _cost:u_R, Value: 0.029833471229469166
  _cost:x_F, Value: 919.3862153636283
  _cost:x_Q, Value: 0.332199696152417
  _ctrlr:horizon, Value: 25
  _model:hidden_size_1, Value: 52
  _model:hidden_size_2, Value: 144
  _model:hidden_size_3, V

use_cuda=True
MLP Using Cuda
hidden_sizes= [52, 144, 134, 128]
100%|██████████| 50/50 [02:19<00:00,  2.78s/it]


We can also auto-tune a particular surrogate class

In [6]:
tuner = PipelineTuner(surrogate_mode="autotune", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result3 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

/home/william/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-30_04:20:41_280303
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided

Evaluating Cfg:
Configuration:
  _MLP:hidden_size_1, Value: 136
  _MLP:hidden_size_2, Value: 136
  _MLP:hidden_size_3, Value: 136
  _MLP:lr, Value: 0.003162277660168379
  _MLP:n_hidden_layers, Value: '3'
  _MLP:nonlintype, Value: 'sigmoid'
  model, Value: 'MLP'

use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:23<00:00,  1.67s/it]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0315



Model Score  0.03151856053630189
Evaluating Cfg:
Configuration:
  _MLP:hidden_size_1, Value: 76
  _MLP:hidden_size_2, Value: 76
  _MLP:lr, Value: 0.056234132519034884
  _MLP:n_hidden_layers, Value: '2'
  _MLP:nonlintype, Value: 'selu'
  model, Value: 'MLP'

use_cuda=True
MLP Using Cuda
hidden_sizes= [76, 76]
100%|██████████| 50/50 [01:04<00:00,  1.30s/it]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 148.436885 sec, available: 0.000010 sec)


...


True Dynamics Cost:  200.0
True Dynamics Final State:  [ 4.22528926 -0.19461106 -0.52535292  0.2896695 ]
use_cuda=True
MLP Using Cuda
hidden_sizes= [187, 140, 194]
100%|██████████| 50/50 [01:50<00:00,  2.20s/it]
Simulating Surrogate Trajectory: 
  2%|▏         | 4/200 [00:00<00:06, 32.66it/s]

/home/william/proj/autompc_dev/autompc/control/ilqr.py:535: RuntimeWarning: invalid value encountered in double_scalars
  if (obj - new_obj) / (-expect_cost_reduction) > ls_cost_threshold:


 99%|█████████▉| 198/200 [00:06<00:00, 31.80it/s]
Surrogate Cost:  200.0
Surrogate Final State:  [ 3.12495379 -0.00510427  0.89749389  0.12104042]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:05<00:00, 33.17it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 122.400830 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 6
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 18862.36 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 18721.47 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  _cost:dx_F, Value: 2075.0627121841476
  _cost:dx_Q, Value: 0.003439168248164087
  _cost:omega_F, Value: 0.08071656763578268
  _cost:omega_Q, Value: 0.0010268426557017


True Dynamics Cost:  200.0
True Dynamics Final State:  [ 3.13851311e+00 -1.40604318e-03  0.00000000e+00  0.00000000e+00]
use_cuda=True
MLP Using Cuda
hidden_sizes= [109, 132, 128, 138]
100%|██████████| 50/50 [02:19<00:00,  2.78s/it]


Finally, we can auto-select and auto-tune the surrogate mode class

In [7]:
tuner = PipelineTuner(surrogate_mode="autoselect", surrogate_split=0.5)

controller, tune_result4 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

/home/william/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-30_12:12:17_059821
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided

Evaluating Cfg:
Configuration:
  _ApproximateGP:induce_count, Value: 125
  model, Value: 'ApproximateGP'

Cuda is used for GPytorch


100%|██████████| 5/5 [00:12<00:00,  2.44s/it]
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.2232


Model Score  0.223215880077073
Evaluating Cfg:
Configuration:
  _ARX:history, Value: 3
  model, Value: 'ARX'



INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 13.067177 sec, available: 0.000010 sec)


...


True Dynamics Cost:  200.0
True Dynamics Final State:  [ 3.13944610e+00 -1.76981073e-03  2.12340494e-01 -1.75190553e-02]
use_cuda=True
MLP Using Cuda
hidden_sizes= [38, 128]
100%|██████████| 50/50 [01:25<00:00,  1.71s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:46<00:00,  4.29it/s]
Surrogate Cost:  67.0
Surrogate Final State:  [-0.01156184  0.012345   -0.15131519 -0.07409573]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:32<00:00,  6.13it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 164.019650 sec, available: 0.000010 sec)



True Dynamics Cost:  77.0
True Dynamics Final State:  [ 0.00110395 -0.23028481  0.09368596 -0.24102533]
use_cuda=True
MLP Using Cuda
hidden_sizes= [52, 101, 102]
100%|██████████| 50/50 [01:48<00:00,  2.18s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:12<00:00, 15.59it/s]
Surrogate Cost:  49.0
Surrogate Final State:  [-0.01015628 -0.01518392  0.09255421 -0.08793458]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:14<00:00, 13.93it/s]

INFO:smac.intensification.intensification.Intensifier:Challenger (49.0000) is better than incumbent (55.0000) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  _cost:dx_F : 7250.118351680813 -> 7990.334273749601
INFO:smac.intensification.intensification.Intensifier:  _cost:dx_Q : 0.02464416797858498 -> 0.026390969205282777
INFO:smac.intensification.intensification.Intensifier:  _cost:omega_F : 0.002262755815018774 -> 0.003037242338892879
INFO:smac.intensification.intensification.Intensifier:  _cost:omega_Q : 0.001466703436222033 -> 0.0014466851874062442
INFO:smac.intensification.intensification.Intensifier:  _cost:theta_F : 8.67523980351373 -> 2.605175947101952
INFO:smac.intensification.intensification.Intensifier:  _cost:theta_Q : 0.0012632336059377586 -> 0.0011604314644478712
INFO:smac.intensification.intensification.Intensifier:  _cost:u_R : 0.013731973451645858 -> 0.014255915736498532
INFO:


True Dynamics Cost:  41.0
True Dynamics Final State:  [-1.56160338e-10  4.50397082e-10  1.00845850e-01 -1.96410416e-10]
use_cuda=True
MLP Using Cuda
hidden_sizes= [52, 101, 102]
100%|██████████| 50/50 [01:50<00:00,  2.21s/it]


Finally, we can pre-train and select our own surrogate.

In [8]:
from autompc.tuning import ModelTuner
from autompc.evaluation import HoldoutModelEvaluator

sysid_trajs = trajs[:250]
surr_trajs = trajs[250:]

model_evaluator = HoldoutModelEvaluator(system=system,trajs=surr_trajs,rng=np.random.default_rng(100),
                                        holdout_prop=0.25, metric="rmse")
model_tuner = ModelTuner(system, model_evaluator)
model_tuner.add_model_factory(surrogate_factory)

surrogate_model, _ = model_tuner.run(rng=np.random.default_rng(100), n_iters = 1)

tuner = PipelineTuner(surrogate_mode="pretrain")

controller, tune_result5 = tuner.run(pipeline, task, sysid_trajs, n_iters=100, surrogate=surrogate_model,
                                    truedyn=benchmark.dynamics)

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-30_18:41:12_510023
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 1 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


Evaluating Cfg:
Configuration:
  _MLP:hidden_size_1, Value: 136
  _MLP:hidden_size_2, Value: 136
  _MLP:hidden_size_3, Value: 136
  _MLP:lr, Value: 0.003162277660168379
  _MLP:n_hidden_layers, Value: '3'
  _MLP:nonlintype, Value: 'sigmoid'
  model, Value: 'MLP'

use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:23<00:00,  1.66s/it]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0304
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 0
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 1 / 1.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 1 / 1.0
INFO:smac.stats.stats.Stats:#Configurations: 1
INFO:smac.stats.stats.Stats:Used wallclock time: 83.10 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 83.09 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  _MLP:hidden_size_1, Value: 136
  _MLP:hidden_size_2, Value: 136
  _MLP:hidden_size_3, Value: 13


Model Score  0.030380426310064948
use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:51<00:00,  2.22s/it]


TypeError: run() missing 1 required positional argument: 'rng'

## Decoupled Tuning

The above examples is full pipeline tuning, which searches the configuration space of all components simultaneously.  Alternatively, we can take a decoupled tuning approach, where the model is first tuned based on prediction accuracy, then the objective function and optimizer are tuned.

First, we must tune the model

In [ ]:
from autompc.tuning import ModelTuner
from autompc.evaluation import HoldoutModelEvaluator

model_evaluator = HoldoutModelEvaluator(holdout_prop=0.25, metric="rmse", trajs=trajs, system=system,
                                       rng=np.random.default_rng(100))
model_tuner = ModelTuner(system, model_evaluator)
model_tuner.add_model_factory(model_factory)

model, model_tune_result = model_tuner.run(rng=np.random.default_rng(100), n_iters=2)

Now that we have tuned the model, we can create a pipeline with the pre-tuned model, and then run pipeline tuning.

In [ ]:
pipeline_fixed_model = ampc.Pipeline(system, model, iLQRFactory, QuadCostFactory)
controller6, tune_result6 = tuner.run(pipeline, task, trajs, n_iters=100, truedyn=benchmark.dynamics)   